In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle as pkl
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
import keras
import keras_metrics as km
from keras import initializers

from keras.callbacks import TensorBoard
from keras import regularizers
from custom_callbacks import LrFinder
from custom_callbacks import CycleLearner
from custom_callbacks import reset_weights
from keras.callbacks import ModelCheckpoint
from time import time
from keras.layers import Input, Dense,Dropout,BatchNormalization,LSTM,GRU,Bidirectional,Conv2D, MaxPool2D, Flatten, GlobalAvgPool2D, GlobalMaxPool2D,merge,CuDNNGRU,CuDNNLSTM
from keras.models import Model,Sequential
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.optimizers import Adagrad
from keras.optimizers import Adamax
from keras.optimizers import Nadam
from keras.optimizers import RMSprop
from keras import backend as K
import math
import json
import keras.backend as K
from keras.layers import Lambda

# Get the dictionary containing each metric and the loss for each epoch

train_filepath = "train_workspace/"
animalList = ["Human","Pig","Chicken","Rat","Mouse","Dog"]
levelList = ["Strict","Relaxed","Intermediate"]
dataList = ["Ohnologs","No-Ohnologs","Paralogs"]
dataNameList = ["ohnologs","no-ohnologs","paralog"]
sns.set_style("darkgrid")


Using TensorFlow backend.


In [2]:
working_level = "Strict"
kmer_chosen = 8
type_chosen = "cdna"

dataset_complete = pd.read_pickle(train_filepath + working_level + "/datasets/2_no_seq_paralog_dataset_complete-" + str(kmer_chosen) + "-" + type_chosen + ".pkl")
dataset_complete_soft = pd.read_pickle(train_filepath + working_level + "/datasets/3_no_seq_paralog_dataset_complete-" + str(kmer_chosen) + "-" + type_chosen + ".pkl")

In [3]:
new_classes = []
for index, row in dataset_complete_soft.iterrows():
    if(row["Is_Ohnolog"] == 1):
        new_classes.append(1)
    else:
        if(row["Is_Paralog"] == 0):
            new_classes.append(0)
        else:
            new_classes.append(2)
dataset_complete_soft["Dup_Class"] = new_classes
dataset_complete_soft.head()

,Specie_Chicken,Specie_Dog,Specie_Human,Specie_Mouse,Specie_Pig,Specie_Rat,Is_Ohnolog,Is_Paralog,Sequence-1 GC,Sequence-2 Length,...,Aligment Length_Total,Nr Mismatch_Total,Nr Gap Open_Total,Evalue_Total,Bit Score_Total,Nr Hits,Seq-2-Biotype_protein_coding,Sequence 1 Numerical,Sequence 2 Numerical,Dup_Class
0,0,0,1,0,0,0,1,0.0,0.429090,0.164613,...,0.046193,0.056091,0.058824,0.457654,0.012462,0.008333,0,"0.5,1.5,1.5,1.5,0.5,0.5,-1.5,-1.5,0.5,1.5,0.5,...","-1.5,0.5,1.5,-1.5,1.5,0.5,1.5,1.5,1.5,1.5,0.5,...",1
1,0,0,1,0,0,0,1,0.0,0.441268,0.142236,...,0.015315,0.012044,0.011272,0.613371,0.008681,0.013235,1,"0.5,0.5,1.5,1.5,1.5,1.5,0.5,0.5,0.5,-1.5,-1.5,...","-1.5,1.5,-1.5,0.5,-1.5,-1.5,1.5,-1.5,0.5,1.5,0...",1
2,0,0,1,0,0,0,1,0.0,0.357972,0.068582,...,0.049161,0.055012,0.027174,0.171352,0.018926,0.003922,1,"-1.5,0.5,1.5,0.5,-1.5,-1.5,0.5,0.5,1.5,0.5,1.5...","-1.5,0.5,-1.5,1.5,-1.5,-1.5,1.5,0.5,1.5,-1.5,-...",1
3,0,0,1,0,0,0,1,0.0,0.684226,0.161686,...,0.011033,0.008087,0.007246,0.464210,0.006696,0.019118,0,"0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,-1.5,0.5,0...","0.5,-1.5,-1.5,0.5,-1.5,-1.5,0.5,0.5,0.5,-1.5,0...",1
4,0,0,1,0,0,0,1,0.0,0.402629,0.104322,...,0.016728,0.012836,0.021739,0.140627,0.008341,0.003922,0,"0.5,0.5,0.5,1.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0....","-1.5,0.5,0.5,0.5,-1.5,1.5,1.5,0.5,1.5,0.5,0.5,...",1


In [4]:
dataset_complete_soft["Sequence 1 Numerical"].values[0].split(",")[-1:]

['']

In [5]:
a = ["A","B","C"]
a[:-1]

['A', 'B']

In [6]:
def get_numerical_embeddings(df):
    sequence_1_array = np.ndarray(shape = (len(df),), dtype=object)
    sequence_2_array = np.ndarray(shape = (len(df),), dtype=object)
    i = 0
    for index,row in df.iterrows():
        numerical_1_list = row["Sequence 1 Numerical"].split(",")[:-1]
        numerical_1_list = list(map(float, numerical_1_list))
    
        numerical_2_list = row["Sequence 2 Numerical"].split(",")[:-1]
        numerical_2_list = list(map(float, numerical_2_list))
        
        sequence_1_array[i] = np.array(numerical_1_list).reshape(len(numerical_1_list),1)
        sequence_2_array[i] = np.array(numerical_2_list).reshape(len(numerical_2_list),1)
        i += 1
    
    return (sequence_1_array,sequence_2_array)

In [7]:
new_classes = []
for index, row in dataset_complete.iterrows():
    if(row["Is_Ohnolog"] == 1):
        new_classes.append(1)
    else:
        if(row["Is_Paralog"] == 0):
            new_classes.append(0)
        else:
            new_classes.append(2)
dataset_complete["Dup_Class"] = new_classes
dataset_complete.head()

,Specie_Chicken,Specie_Dog,Specie_Human,Specie_Mouse,Specie_Pig,Specie_Rat,Is_Ohnolog,Is_Paralog,Sequence-1 GC,Sequence-2 Length,...,Aligment Length_Total,Nr Mismatch_Total,Nr Gap Open_Total,Evalue_Total,Bit Score_Total,Nr Hits,Seq-2-Biotype_protein_coding,Sequence 1 Numerical,Sequence 2 Numerical,Dup_Class
0,0,0,1,0,0,0,1,0.0,0.429090,0.163814,...,0.046193,0.056091,0.058824,0.457654,0.012462,0.008333,0,"0.5,1.5,1.5,1.5,0.5,0.5,-1.5,-1.5,0.5,1.5,0.5,...","-1.5,0.5,1.5,-1.5,1.5,0.5,1.5,1.5,1.5,1.5,0.5,...",1
1,0,0,1,0,0,0,1,0.0,0.441268,0.141415,...,0.015315,0.012044,0.011272,0.613371,0.008681,0.013235,1,"0.5,0.5,1.5,1.5,1.5,1.5,0.5,0.5,0.5,-1.5,-1.5,...","-1.5,1.5,-1.5,0.5,-1.5,-1.5,1.5,-1.5,0.5,1.5,0...",1
2,0,0,1,0,0,0,1,0.0,0.357972,0.067690,...,0.049161,0.055012,0.027174,0.171352,0.018926,0.003922,1,"-1.5,0.5,1.5,0.5,-1.5,-1.5,0.5,0.5,1.5,0.5,1.5...","-1.5,0.5,-1.5,1.5,-1.5,-1.5,1.5,0.5,1.5,-1.5,-...",1
3,0,0,1,0,0,0,1,0.0,0.684226,0.160883,...,0.011033,0.008087,0.007246,0.464210,0.006696,0.019118,0,"0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,-1.5,0.5,0...","0.5,-1.5,-1.5,0.5,-1.5,-1.5,0.5,0.5,0.5,-1.5,0...",1
4,0,0,1,0,0,0,1,0.0,0.402629,0.103464,...,0.016728,0.012836,0.021739,0.140627,0.008341,0.003922,0,"0.5,0.5,0.5,1.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0....","-1.5,0.5,0.5,0.5,-1.5,1.5,1.5,0.5,1.5,0.5,0.5,...",1


In [8]:
blast_columns = ["Percent Identical Matches","Aligment Length","Nr Mismatch","Nr Gap Open","Evalue","Bit Score"]

e_values_columns = ["Evalue_Total","Evalue_High","Evalue_Medium","Evalue_Low"]

blast_types = ["High","Medium","Low","Total"]
blast_col_Total = [i + "_Total" for i in blast_columns] 
blast_col_High = [i + "_High" for i in blast_columns]
blast_col_Medium = [i + "_Medium" for i in blast_columns]
blast_col_Low = [i + "_Low" for i in blast_columns]

blast_cols_levels = blast_col_High + blast_col_Medium + blast_col_Low
total_columns = (blast_col_High + blast_col_Medium + blast_col_Low + blast_col_Total + ["Nr Hits"])

non_training_meta_features = ["Sequence-1","Sequence-2","Sequence-1 Id","Sequence-1-Transcript Id","Sequence-2 Id","Sequence-2-Transcript Id","Sequence-1-Transcript-Version","Sequence-2-Transcript-Version"]

sequence_1_metadata = ["Sequence-1 GC","Sequence-1 Length","Sequence-1-Chromosome","Seq-1-Biotype_protein_coding"]
sequence_2_metadata = ["Sequence-2 GC","Sequence-2 Length","Sequence-2-Chromosome","Seq-2-Biotype_protein_coding"]

sequence_1_metadata_diferential = ["Sequence-1 GC","Sequence-1 Length"]
sequence_2_metadata_diferential = ["Sequence-2 GC","Sequence-2 Length"]

sequence_1_metadata_categorical = ["Sequence-1-Chromosome","Seq-1-Biotype_protein_coding"]
sequence_2_metadata_categorical = ["Sequence-2-Chromosome","Seq-2-Biotype_protein_coding"]

species_metadata = ["Specie_Chicken","Specie_Dog","Specie_Human","Specie_Mouse","Specie_Pig","Specie_Rat"]

emb_size = 100
if(type_chosen == "cdna2"):
    emb_size = 200
embedding_1_cols = ["Embedding1_" + str(i) for i in range(0,emb_size)]
embedding_2_cols = ["Embedding2_" + str(i) for i in range(0,emb_size)]

In [9]:
def get_train_val_test(df):
    df.sample(frac=1,random_state=7)
    df_train, df_test = train_test_split(df, test_size=0.2,random_state=9,stratify=df["Is_Ohnolog"])
    df_train, df_val = train_test_split(df_train, test_size=0.2,random_state=3,stratify=df_train["Is_Ohnolog"])
    return (df_train,df_val,df_test)

In [10]:
def get_train_val_test_soft(df):
    df.sample(frac=1,random_state=7)
    df_train, df_test = train_test_split(df, test_size=0.2,random_state=9,stratify=df["Dup_Class"])
    df_train, df_val = train_test_split(df_train, test_size=0.2,random_state=3,stratify=df_train["Dup_Class"])
    return (df_train,df_val,df_test)

In [11]:
def save_log(log,name):
    log_dict = log.history    
    json.dump(log_dict, open(train_filepath + working_level + "/model_run_history/" + name + ".json", 'w'))

In [12]:
def get_log(name):
     return json.load(open(train_filepath + working_level + "/model_run_history/" + name + ".json", 'r'))

In [13]:
class EpochLogger(keras.callbacks.Callback):
    def __init__(self, display):
        self.step = 0
        self.display = display
        self.metric_cache = {}        

    def on_epoch_end(self, epoch, logs={}):
        self.step += 1
        for k in self.params['metrics']:
            if k in logs:
                self.metric_cache[k] = self.metric_cache.get(k, 0) + logs[k]
        if self.step % self.display == 0:
            metrics_log_default = ''            
            metrics_log_main = ''            
            metrics_log_aux = ''            
            for (k, v) in self.metric_cache.items():
                val = v / self.display             
                if("main" in k):
                    if abs(val) > 1e-3:
                        metrics_log_main += ' - %s: %.4f' % (k, val)
                        continue      
                    else:
                        metrics_log_main += ' - %s: %.4e' % (k, val)
                        continue                    
                        
                if("aux" in k): 
                    if abs(val) > 1e-3:
                        metrics_log_aux += ' - %s: %.4f' % (k, val)
                        continue      
                    else:
                        metrics_log_aux += ' - %s: %.4e' % (k, val)
                        continue      
                        
                if abs(val) > 1e-3:
                        metrics_log_default += ' - %s: %.4f' % (k, val)
                else:
                        metrics_log_default += ' - %s: %.4e' % (k, val)                        
                    
            print('step: {}/{} ... {}'.format(self.step,
                                          self.params['epochs'],
                                          metrics_log_default + "\n" + metrics_log_main + "\n" + metrics_log_aux))
            self.metric_cache.clear()

## RNN

In [14]:
def get_tensor_embedding_RNN_layer(input_embedding):    
    emb_x = GRU(128,activation="relu",use_bias=False,recurrent_regularizer=regularizers.l1(0.0005),kernel_regularizer=regularizers.l1(0.005))(input_embedding)  
    emb_x = BatchNormalization()(emb_x)    
    return emb_x

def level_siames_merge_RNN_layer(df_meta_input,name,bs):    
    
    input_embedding_1 = Input(shape=(None,1), name='embedding_input_1')        
    input_embedding_2 = Input(shape=(None,1), name='embedding_input_2')        
    
    emb_x_1 = get_tensor_embedding_RNN_layer(input_embedding_1)    
    emb_x_2 = get_tensor_embedding_RNN_layer(input_embedding_2)    
    
    L1_layer = Lambda(lambda tensors:(tensors[0] - tensors[1]))
    L1_distance = L1_layer([emb_x_1, emb_x_2])
   
    
    emb_x_out = Dense(1, activation='sigmoid',name="aux")(L1_distance)                
    
    meta_input = Input(shape=(len(df_meta_input.columns),), name='meta_input')    
    
    x = keras.layers.concatenate([L1_distance,meta_input])
    x = Dense(256, activation='relu',use_bias=False,kernel_regularizer=regularizers.l1(0.0005))(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu',use_bias=False,kernel_regularizer=regularizers.l1(0.0005))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu',use_bias=False,kernel_regularizer=regularizers.l1(0.0005))(x)   
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu',use_bias=False,kernel_regularizer=regularizers.l1(0.0005))(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu',kernel_regularizer=regularizers.l1(0.0005))(x)    
    x = Dropout(0.1)(x)
    x = Dense(16, activation='relu',use_bias=False,kernel_regularizer=regularizers.l1(0.0005))(x)
    x = BatchNormalization()(x)
    predictions = Dense(1, activation='sigmoid',name="main")(x)
        
    model_created = Model(inputs=[input_embedding_1,input_embedding_2,meta_input], outputs=[predictions,emb_x_out])
    model_created.Name = name
    return model_created

## Siames

In [15]:
def get_tensor_embedding_layer(input_embedding):    
    emb_x = Dense(512, activation='elu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(input_embedding)       
    emb_x = BatchNormalization()(emb_x)
    emb_x = Dropout(0.4)(emb_x)    
    emb_x = Dense(256,use_bias = False, activation='elu',kernel_regularizer=regularizers.l1(0.0005))(emb_x)            
    emb_x = BatchNormalization()(emb_x)    
    emb_x = Dropout(0.4)(emb_x)    
    emb_x = Dense(128,use_bias = False, activation='elu',kernel_regularizer=regularizers.l1(0.0005))(emb_x)            
    emb_x = BatchNormalization()(emb_x)    
    emb_x = Dense(64, use_bias = False, activation='elu',kernel_regularizer=regularizers.l1(0.0005))(emb_x)            
    emb_x = BatchNormalization()(emb_x)    
    emb_x = Dense(32, use_bias = False, activation='elu',kernel_regularizer=regularizers.l1(0.0005))(emb_x)                
    emb_x = BatchNormalization()(emb_x)    
    return emb_x


In [16]:
def level_siames_merge_layer(df_meta_input,df_embeddings_1,df_embeddings_2,name,bs):    
    
    input_embedding_1 = Input(shape=(len(df_embeddings_1.columns),), name='embedding_input_1')        
    input_embedding_2 = Input(shape=(len(df_embeddings_2.columns),), name='embedding_input_2')        
    
    emb_x_1 = get_tensor_embedding_layer(input_embedding_1)    
    emb_x_2 = get_tensor_embedding_layer(input_embedding_2)    
    
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([emb_x_1, emb_x_2])
    
    emb_x_out = Dense(1, activation='sigmoid',name="aux")(L1_distance)                
    
    meta_input = Input(shape=(len(df_meta_input.columns),), name='meta_input')    
    
    x = keras.layers.concatenate([L1_distance,meta_input])
    x = Dense(128, activation='relu',use_bias=False)(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)   
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(x)    
    x = Dropout(0.1)(x)
    x = Dense(8, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.0005))(x)
    x = BatchNormalization()(x)
    predictions = Dense(1, activation='sigmoid',name="main")(x)
        
    model_created = Model(inputs=[input_embedding_1,input_embedding_2,meta_input], outputs=[predictions,emb_x_out])
    model_created.Name = name
    return model_created

In [17]:
def level_siames_merge_layer_soft(df_meta_input,df_embeddings_1,df_embeddings_2,name,bs):    
    
    input_embedding_1 = Input(shape=(len(df_embeddings_1.columns),), name='embedding_input_1')        
    input_embedding_2 = Input(shape=(len(df_embeddings_2.columns),), name='embedding_input_2')        
    
    emb_x_1 = get_tensor_embedding_layer(input_embedding_1)    
    emb_x_2 = get_tensor_embedding_layer(input_embedding_2)    
    
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([emb_x_1, emb_x_2])
    
    emb_x_out = Dense(3, activation='softmax',name="aux")(L1_distance)                
    
    meta_input = Input(shape=(len(df_meta_input.columns),), name='meta_input')    
    
    x = keras.layers.concatenate([L1_distance,meta_input])
    x = Dense(128, activation='relu',use_bias=False)(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)   
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(x)    
    x = Dropout(0.1)(x)
    x = Dense(8, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.0005))(x)
    x = BatchNormalization()(x)
    predictions = Dense(3, activation='softmax',name="main")(x)
        
    model_created = Model(inputs=[input_embedding_1,input_embedding_2,meta_input], outputs=[predictions,emb_x_out])
    model_created.Name = name
    return model_created


In [18]:
def euclidean_distance(x,y):    
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)



In [19]:
def fit_model_siames(train_x,train_emb_x_1,train_emb_x_2,train_y,val_x,val_emb_x_1,val_emb_x_2,val_y,model_train,n_epochs,optimizer,batchsize,loss_weigths,verb):
    tensorboard = TensorBoard(log_dir= train_filepath + working_level + "/board_logs/" + model_train.Name + "-{}".format(time()))
    checkpoint = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-check-{{epoch:02d}}-{{val_main_acc:.2f}}.hdf5".format(model_train.Name),save_weights_only=True,  period = int(n_epochs/5))
    best_model_save = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-best.hdf5".format(model_train.Name), monitor='val_main_acc', save_weights_only=True, save_best_only=True, mode='max')
    logger = EpochLogger(display=25)

    model_train.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy',km.binary_recall()],loss_weights=loss_weigths)    
    return model_train.fit([train_emb_x_1,train_emb_x_2,train_x], y = [train_y,train_y],verbose = verb,validation_data=([val_emb_x_1,val_emb_x_2,val_x],[val_y,val_y]),epochs = n_epochs,batch_size=batchsize,callbacks = [tensorboard,checkpoint,best_model_save,logger])  # starts training


def fit_model_siames_soft(train_x,train_emb_x_1,train_emb_x_2,train_y,val_x,val_emb_x_1,val_emb_x_2,val_y,model_train,n_epochs,optimizer,batchsize,loss_weigths,verb):
    tensorboard = TensorBoard(log_dir=train_filepath + working_level + "/board_logs/" + model_train.Name + "-{}".format(time()))
    checkpoint = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-check-{{epoch:02d}}-{{val_main_acc:.2f}}.hdf5".format(model_train.Name),save_weights_only=True,  period = int(n_epochs/5))
    best_model_save = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-best.hdf5".format(model_train.Name), monitor='val_main_acc', save_weights_only=True, save_best_only=True, mode='max')
    logger = EpochLogger(display=25)

    model_train.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy',km.sparse_categorical_recall()],loss_weights=loss_weigths)    
    return model_train.fit([train_emb_x_1,train_emb_x_2,train_x], y = [train_y,train_y],verbose = verb,validation_data=([val_emb_x_1,val_emb_x_2,val_x],[val_y,val_y]),epochs = n_epochs,batch_size=batchsize,callbacks = [tensorboard,checkpoint,best_model_save,logger])  # starts training

In [24]:
def train_generator(x_train_meta,x_train_1,x_train_2,y_train):
    i = 0
    while True:
        if(i == len(x_train_meta)):
            i = 0
        x_train_out = x_train_meta[i].reshape(1,len(x_train_meta[i]))
                
        x_train_1_out = x_train_1[i].reshape(1,len(x_train_1[i]),1)
        x_train_2_out = x_train_2[i].reshape(1,len(x_train_2[i]),1)
        y_train_out = y_train[i].reshape(1,1)
        i += 1
        yield [x_train_1_out,x_train_2_out,x_train_out], [y_train_out,y_train_out]

In [38]:
def val_generator(x_val_meta,x_val_1,x_val_2,y_val):
    i = 0
    while True:
        if(i == len(x_val_meta)):
            i = 0
        x_val_out = x_val_meta[i].reshape(1,len(x_val_meta[i]))
                
        x_val_1_out = x_val_1[i].reshape(1,len(x_val_1[i]),1)
        x_val_2_out = x_val_2[i].reshape(1,len(x_val_2[i]),1)
        y_val_out = y_val[i].reshape(1,1)
        i += 1
        yield [x_val_1_out,x_val_2_out,x_val_out], [y_val_out,y_val_out]

In [42]:
def fit_model_siames_v2(train_x,train_emb_x_1,train_emb_x_2,train_y,val_x,val_emb_x_1,val_emb_x_2,val_y,model_train,n_epochs,optimizer,batchsize,loss_weigths,verb):
    tensorboard = TensorBoard(log_dir= train_filepath + working_level + "/board_logs/" + model_train.Name + "-{}".format(time()))
    checkpoint = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-check-{{epoch:02d}}-{{val_main_acc:.2f}}.hdf5".format(model_train.Name),save_weights_only=True,  period = int(n_epochs/5))
    best_model_save = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-best.hdf5".format(model_train.Name), monitor='val_main_acc', save_weights_only=True, save_best_only=True, mode='max')
    logger = EpochLogger(display=25)

    model_train.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy',km.binary_recall()],loss_weights=loss_weigths)    
    return model_train.fit_generator(train_generator(train_x,train_emb_x_1,train_emb_x_2,train_y),validation_data=val_generator(val_x,val_emb_x_1,val_emb_x_2,val_y),verbose = verb,epochs = n_epochs,steps_per_epoch=len(train_x)/batchsize,validation_steps=len(val_x)/batchsize ,callbacks = [tensorboard,checkpoint,best_model_save,logger])  # starts training

In [ ]:
species_siames_dict = {}
for specie in species_metadata:
    dataset_complete_no_species = dataset_complete[dataset_complete[specie] == 0]    
    dataset_complete_species = dataset_complete[dataset_complete[specie] == 1]    
    
    df_train,df_val,df_test = get_train_val_test(dataset_complete_no_species)
    
    df_train_x = df_train.drop(["Is_Ohnolog","Is_Paralog","Dup_Class","Sequence 1 Numerical","Sequence 2 Numerical"],axis=1)    

    df_train_emb_x_1, df_train_emb_x_2 = get_numerical_embeddings(df_train)    
    df_train_y = df_train["Is_Ohnolog"]

    df_val_x = df_val.drop(["Is_Ohnolog","Is_Paralog","Dup_Class","Sequence 1 Numerical","Sequence 2 Numerical"],axis=1)    

    df_val_emb_x_1,df_val_emb_x_2 = get_numerical_embeddings(df_val)    
    df_val_y = df_val["Is_Ohnolog"]

    df_test_x = df_test.drop(["Is_Ohnolog","Is_Paralog","Dup_Class","Sequence 1 Numerical","Sequence 2 Numerical"],axis=1)    

    df_test_emb_x_1,df_test_emb_x_2 =  get_numerical_embeddings(df_test)    
    df_test_y = df_test["Is_Ohnolog"]
    
    
    model = level_siames_merge_RNN_layer(df_train_x,"Level_Siames_RNN" + specie,128)
    
    log = fit_model_siames_v2(df_train_x.values,df_train_emb_x_1,df_train_emb_x_2,df_train_y.values,df_val_x.values,df_val_emb_x_1,df_val_emb_x_2,df_val_y.values,model,50,Adamax(),128,[1,0.3],2)
    
    break
    df_test_species_x = dataset_complete_species.drop(["Is_Ohnolog","Is_Paralog","Dup_Class","Sequence 1 Numerical","Sequence 2 Numerical"],axis=1)    

    df_test_species_emb_x_1 = dataset_complete_species[embedding_1_cols]
    df_test_species_emb_x_2 = dataset_complete_species[embedding_2_cols]
    df_test_species_y = dataset_complete_species["Is_Ohnolog"]
    
    species_siames_dict[specie] = {}
    
    print("Evaluando especie Test: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_emb_x_1.values,df_test_emb_x_2.values,df_test_x.values],[df_test_y.values,df_test_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_siames_dict[specie]["Test"] = metrics[5]
            
        
    print("Evaluando especie sin balancear: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x_1.values,df_test_species_emb_x_2.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_siames_dict[specie]["Sin Balanceo"] = metrics[5]
    
    df_species_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 1]
    df_species_no_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 0]
    
    if(len(df_species_ohnologs)>len(df_species_no_ohnologs)):
        df_species_new = df_species_no_ohnologs.append(df_species_ohnologs.sample(len(df_species_no_ohnologs)))        
    else:
        df_species_new = df_species_ohnologs.append(df_species_no_ohnologs.sample(len(df_species_ohnologs)))

    if(len(df_species_new) == 0):
        df_species_new = df_species_ohnologs        
                
    df_test_species_x = df_species_new.drop(["Is_Ohnolog","Is_Paralog","Dup_Class","Sequence 1 Numerical","Sequence 2 Numerical"],axis=1)    

    df_test_species_emb_x_1 =  df_species_new[embedding_1_cols]
    df_test_species_emb_x_2 =  df_species_new[embedding_2_cols]
    df_test_species_y = df_species_new["Is_Ohnolog"]    
    
    print("Evaluando especie balanceada: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x_1.values,df_test_species_emb_x_2.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_siames_dict[specie]["Balanceadas"] = metrics[5]   

Epoch 1/50
 - 608s - loss: 1.9905 - main_loss: 0.6964 - aux_loss: 0.7015 - main_acc: 0.5083 - main_recall: 0.8780 - aux_acc: 0.5250 - aux_recall: 0.8699 - val_loss: 0.9426 - val_main_loss: 0.6925 - val_aux_loss: 0.7246 - val_main_acc: 0.5333 - val_main_recall: 0.5161 - val_aux_acc: 0.4667 - val_aux_recall: 0.4839
Epoch 2/50
 - 605s - loss: 0.9296 - main_loss: 0.6944 - aux_loss: 0.6984 - main_acc: 0.4917 - main_recall: 0.3162 - aux_acc: 0.4417 - aux_recall: 0.3162 - val_loss: 0.9288 - val_main_loss: 0.6941 - val_aux_loss: 0.7014 - val_main_acc: 0.4667 - val_main_recall: 0.0000e+00 - val_aux_acc: 0.4667 - val_aux_recall: 0.0000e+00
Epoch 3/50
 - 579s - loss: 0.9257 - main_loss: 0.6929 - aux_loss: 0.6949 - main_acc: 0.5417 - main_recall: 0.0000e+00 - aux_acc: 0.5417 - aux_recall: 0.0000e+00 - val_loss: 0.9301 - val_main_loss: 0.6964 - val_aux_loss: 0.6982 - val_main_acc: 0.4667 - val_main_recall: 0.0000e+00 - val_aux_acc: 0.4667 - val_aux_recall: 0.0000e+00
Epoch 4/50
 - 600s - loss: 0.92

In [ ]:
species_siames_cross_dict = {}
for specie in species_metadata:
    dataset_complete_no_species = dataset_complete[dataset_complete[specie] == 0]    
    dataset_complete_species = dataset_complete[dataset_complete[specie] == 1]    
        
    df_no_species = dataset_complete_no_species[dataset_complete_no_species[specie] == 0]
    
    df_train, df_val = train_test_split(df_no_species, test_size=0.2,random_state=9,stratify=df_no_species["Is_Ohnolog"])
    
    df_train_x = df_train.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_train_x = df_train_x.drop(embedding_1_cols,axis=1)
    df_train_x = df_train_x.drop(embedding_2_cols,axis=1)
    
    df_train_emb_x_1 = df_train[embedding_1_cols]
    df_train_emb_x_2 = df_train[embedding_2_cols]

    model = level_siames_merge_layer(df_train_x,df_train_emb_x_1,df_train_emb_x_2,"Siames " + specie,128)        
    specie_metadata_copy = species_metadata[:]
    species_metadata_no_species = specie_metadata_copy.remove(specie)

    for specie_cross in species_metadata_no_species:
        df_no_species = dataset_complete_no_species[dataset_complete_no_species[specie_cross] == 0]
        df_train, df_val = train_test_split(df_no_species, test_size=0.2,random_state=9,stratify=df_no_species["Is_Ohnolog"])

        df_train_x = df_train.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
        df_train_x = df_train_x.drop(embedding_1_cols,axis=1)
        df_train_x = df_train_x.drop(embedding_2_cols,axis=1)

        df_train_emb_x_1 = df_train[embedding_1_cols]
        df_train_emb_x_2 = df_train[embedding_2_cols]
        df_train_y = df_train["Is_Ohnolog"]

        df_val_x = df_val.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
        df_val_x = df_val_x.drop(embedding_1_cols,axis=1)
        df_val_x = df_val_x.drop(embedding_2_cols,axis=1)

        df_val_emb_x_1 = df_val[embedding_1_cols]
        df_val_emb_x_2 = df_val[embedding_2_cols]
        df_val_y = df_val["Is_Ohnolog"]

        print("Entrenando sin especie: " + species)
        log = fit_model_siames(df_train_x.values,df_train_emb_x_1.values,df_train_emb_x_2.values,df_train_y.values,df_val_x.values,df_val_emb_x_1.values,df_val_emb_x_2.values,df_val_y.values,model,200,Adamax(),256,[1,0.2],0)
        
    
    df_test_species_x = dataset_complete_species.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_1_cols,axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_2_cols,axis=1)

    df_test_species_emb_x_1 =  dataset_complete_species[embedding_1_cols]
    df_test_species_emb_x_2 =  dataset_complete_species[embedding_2_cols]
    df_test_species_y = dataset_complete_species["Is_Ohnolog"]
    
    species_siames_cross_dict[specie] = {}
    
    print("Evaluando especie sin balancear: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x_1.values,df_test_species_emb_x_2.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_siames_cross_dict[specie]["Sin Balanceo"] = metrics[5]
    
    df_species_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 1]
    df_species_no_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 0]
    
    if(len(df_species_ohnologs)>len(df_species_no_ohnologs)):
        df_species_new = df_species_no_ohnologs.append(df_species_ohnologs.sample(len(df_species_no_ohnologs)))        
    else:
        df_species_new = df_species_ohnologs.append(df_species_no_ohnologs.sample(len(df_species_ohnologs)))

    if(len(df_species_new) == 0):
        df_species_new = df_species_ohnologs        
        
    df_test_species_x = df_species_new.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_1_cols,axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_2_cols,axis=1)

    df_test_species_emb_x_1 = df_species_new[embedding_1_cols]
    df_test_species_emb_x_2 = df_species_new[embedding_2_cols]
    df_test_species_y = df_species_new["Is_Ohnolog"]                    
    
    print("Evaluando especie balanceada: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x_1.values,df_test_species_emb_x_2.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_cross_dict[specie]["Balanceadas"] = metrics[5]   